In [1]:
# Set to True if running notebook on AWS/EMR
EMR=True
%pwd

'/mnt/workspace/edX-Micro-Master-in-Data-Science/big-data-analytics-using-spark/notebooks/Section2-PCA/PCA/data_preparation'

In [2]:
if not EMR:
    import findspark
    findspark.init()
from pyspark import SparkContext,SparkConf

def create_sc(pyFiles):
    sc_conf = SparkConf()
    sc_conf.setAppName("Weather_PCA")
    sc_conf.set('spark.executor.memory', '10g')
    sc_conf.set('spark.executor.cores', '4')
    sc_conf.set('spark.cores.max', '4')
    sc_conf.set('spark.default.parallelism','10')
    sc_conf.set('spark.logConf', True)
    print(sc_conf.getAll())

    sc = SparkContext(conf=sc_conf,pyFiles=pyFiles)

    return sc 

sc = create_sc(pyFiles=['lib/numpy_pack.py','lib/spark_PCA.py','lib/computeStatistics.py'])

dict_items([('spark.logConf', 'True'), ('spark.default.parallelism', '10'), ('spark.cores.max', '4'), ('spark.app.name', 'Weather_PCA'), ('spark.executor.memory', '10g'), ('spark.executor.cores', '4')])


In [3]:
from pyspark.sql.types import ArrayType,FloatType, DoubleType, StringType, IntegerType
from pyspark.sql.functions import udf
from pyspark.sql import *
sqlContext = SQLContext(sc)

import numpy as np
from lib.computeStatistics import *

In [4]:
file_list=!aws s3 ls mas-dse-open/Weather/by_state/

In [5]:
filenames=[l.split()[3] for l in file_list]
len(filenames)
filenames[36:]

['NY.tgz',
 'NaN.tgz',
 'OH.tgz',
 'OK.tgz',
 'ON.tgz',
 'OR.tgz',
 'PA.tgz',
 'QC.tgz',
 'RI.tgz',
 'SC.tgz',
 'SD.tgz',
 'SK.tgz',
 'STAT_AB.pickle',
 'STAT_AL.pickle',
 'STAT_AR.pickle',
 'STAT_NY.pickle',
 'STAT_NY.pickle.gz',
 'STAT_RI.pickle.gz',
 'TN.tgz',
 'TX.tgz',
 'UT.tgz',
 'VA.tgz',
 'VT.tgz',
 'WA.tgz',
 'WI.tgz',
 'WV.tgz',
 'WY.tgz']

In [6]:
!ls /mnt/workspace/Data/

by_state  US_stations.parquet  US_weather.parquet


In [13]:
!aws s3 cp --recursive s3://mas-dse-open/Weather/by_state/ /mnt/workspace/Data/by_state/

download: s3://mas-dse-open/Weather/by_state/AB.tgz to ../../../../../../Data/by_state/AB.tgz
download: s3://mas-dse-open/Weather/by_state/BC.tgz to ../../../../../../Data/by_state/BC.tgz
download: s3://mas-dse-open/Weather/by_state/CT.tgz to ../../../../../../Data/by_state/CT.tgz
download: s3://mas-dse-open/Weather/by_state/AR.tgz to ../../../../../../Data/by_state/AR.tgz
download: s3://mas-dse-open/Weather/by_state/DC.tgz to ../../../../../../Data/by_state/DC.tgz
download: s3://mas-dse-open/Weather/by_state/DE.tgz to ../../../../../../Data/by_state/DE.tgz
download: s3://mas-dse-open/Weather/by_state/AL.tgz to ../../../../../../Data/by_state/AL.tgz
download: s3://mas-dse-open/Weather/by_state/CO.tgz to ../../../../../../Data/by_state/CO.tgz
download: s3://mas-dse-open/Weather/by_state/AZ.tgz to ../../../../../../Data/by_state/AZ.tgz
download: s3://mas-dse-open/Weather/by_state/GA.tgz to ../../../../../../Data/by_state/GA.tgz
download: s3://mas-dse-open/Weather/by_state/IA.tgz to ../..

In [7]:
data_dir='/mnt/workspace/Data/by_state/'
hdfs_dir='/weather/by_state/'
%cd $data_dir

/mnt/workspace/Data/by_state


In [25]:
from glob import glob
for filename in glob('*.tgz'):
    !tar -xzf $filename

In [22]:
!hdfs dfs -mkdir $hdfs_dir

mkdir: `/weather/by_state': File exists


In [30]:
for filename in glob('*.parquet'):
    hdfs_fullname=hdfs_dir+filename
    !hdfs dfs -copyFromLocal $filename $hdfs_fullname

In [23]:
','.join(state_names)

'AB,AL,AR,AZ,BC,CA,CO,CT,DC,DE,FL,GA,IA,ID,IL,IN,KS,KY,LA,MA,MB,MD,ME,MI,MN,MO,MS,MT,NaN,NB,NC,ND,NE,NH,NJ,NM,NV,NY,OH,OK,ON,OR,PA,QC,RI,SC,SD,SK,TN,TX,UT,VA,VT,WA,WI,WV,WY'

In [9]:
parquet_files=!ls -d /mnt/workspace/Data/by_state/*.parquet
state_names=[a[29:-8] for a in parquet_files]
state_names[35:]

['NM',
 'NV',
 'NY',
 'OH',
 'OK',
 'ON',
 'OR',
 'PA',
 'QC',
 'RI',
 'SC',
 'SD',
 'SK',
 'TN',
 'TX',
 'UT',
 'VA',
 'VT',
 'WA',
 'WI',
 'WV',
 'WY']

In [10]:
def Count_nan(V):
    A=unpackArray(V,data_type=np.float16)
    return int(sum(np.isnan(A)))  # the int() is important, sparksql does not accept numpy ints
Count_nan_udf = udf(Count_nan,IntegerType())

In [16]:
state_names[35:]

['NM',
 'NV',
 'NY',
 'OH',
 'OK',
 'ON',
 'OR',
 'PA',
 'QC',
 'RI',
 'SC',
 'SD',
 'SK',
 'TN',
 'TX',
 'UT',
 'VA',
 'VT',
 'WA',
 'WI',
 'WV',
 'WY']

In [21]:
!ls -l /mnt/workspace/Data/by_state/*.pickle.gz

-rw-rw-r-- 1 hadoop hadoop 28749915 Apr 26 05:25 /mnt/workspace/Data/by_state/STAT_AB.pickle.gz
-rw-rw-r-- 1 hadoop hadoop 28749949 Apr 26 05:32 /mnt/workspace/Data/by_state/STAT_AL.pickle.gz
-rw-rw-r-- 1 hadoop hadoop 29337474 Apr 26 05:41 /mnt/workspace/Data/by_state/STAT_AR.pickle.gz
-rw-rw-r-- 1 hadoop hadoop 31631094 Apr 26 05:54 /mnt/workspace/Data/by_state/STAT_AZ.pickle.gz
-rw-rw-r-- 1 hadoop hadoop 23019414 Apr 26 05:55 /mnt/workspace/Data/by_state/STAT_BC.pickle.gz
-rw-rw-r-- 1 hadoop hadoop 33578888 Apr 26 06:19 /mnt/workspace/Data/by_state/STAT_CA.pickle.gz
-rw-rw-r-- 1 hadoop hadoop 33775498 Apr 26 06:31 /mnt/workspace/Data/by_state/STAT_CO.pickle.gz
-rw-rw-r-- 1 hadoop hadoop 29177399 Apr 26 06:34 /mnt/workspace/Data/by_state/STAT_CT.pickle.gz
-rw-rw-r-- 1 hadoop hadoop 29177399 Apr 26 06:34 /mnt/workspace/Data/by_state/STAT_DC.pickle.gz
-rw-rw-r-- 1 hadoop hadoop 27853245 Apr 26 06:35 /mnt/workspace/Data/by_state/STAT_DE.pickle.gz
-rw-rw-r-- 1 hadoop hadoop 27499127 Apr 

In [20]:
len(state_names)

57

In [ ]:
from pickle import dump

for state in state_names[35:]:
    parquet=state+'.parquet'
    parquet_path = hdfs_dir+'/'+parquet
    print('path=',parquet_path)
    
    df=sqlContext.read.parquet(parquet_path)
    print('dataframe length=',df.cache().count())

    sqlContext.registerDataFrameAsTable(df,'table')

    #find all measurements (should be 6x2 = 12)
    Query="""
    SELECT Measurement,count(Measurement) as count 
    FROM table 
    GROUP BY Measurement
    ORDER BY Measurement
    """
    counts=sqlContext.sql(Query).toPandas()
    measurements=list(counts['Measurement'])
    print(counts)
    print(counts.shape)

    # Remove rows with more than 50 nans
    df=df.withColumn("nan_no", Count_nan_udf(df.Values))
    count_nans=df.select('nan_no').toPandas()
    count_nans.plot.hist('nan_no',bins=100);
    print('before removing >= 50 nans',df.count())
    df=df.filter(df.nan_no < 50)
    print('after removing >= 50 nans',df.count())

    ### This is the main cell, where all of the statistics are computed.
    try:
        STAT=computeStatistics(sqlContext,df)
    except:
        print('error computing statistics for '+state)

    ## Dump STAT and STAT_Descriptions into a pickle file.
    filename='STAT_%s.pickle'%state
    fullname=data_dir+filename
    s3_path='s3://mas-dse-open/Weather/by_state/'
    gzipped_name=fullname+'.gz'
    s3_fullname=s3_path+filename+'.gz'

    print(fullname,s3_fullname)
    dump((STAT,STAT_Descriptions),open(fullname,'wb'))
    !gzip -f $fullname
    
    
    #copy pickle file to S3
    #!aws s3 rm $s3_fullname
    #!aws s3 cp $fullname $s3_fullname

path= /weather/by_state//NM.parquet
dataframe length= 189718
   Measurement  count
0         PRCP  17414
1     PRCP_s20  17414
2         SNOW  17056
3     SNOW_s20  17056
4         SNWD  15558
5     SNWD_s20  15558
6         TMAX  15748
7     TMAX_s20  15748
8         TMIN  15757
9     TMIN_s20  15757
10        TOBS  13326
11    TOBS_s20  13326
(12, 2)
before removing >= 50 nans 189718
after removing >= 50 nans 157360
SELECT * FROM weather
	WHERE measurement = 'SNWD'
SNWD : shape of mdf is  8984
time for SNWD is 47.07102656364441
SELECT * FROM weather
	WHERE measurement = 'TOBS'
TOBS : shape of mdf is  9882
time for TOBS is 47.90512728691101
SELECT * FROM weather
	WHERE measurement = 'SNWD_s20'
SNWD_s20 : shape of mdf is  11554
time for SNWD_s20 is 73.73382449150085
SELECT * FROM weather
	WHERE measurement = 'TOBS_s20'
TOBS_s20 : shape of mdf is  12203
time for TOBS_s20 is 63.62606453895569
SELECT * FROM weather
	WHERE measurement = 'TMIN'
TMIN : shape of mdf is  12623
time for TMIN is

In [27]:
X=STAT['TMAX']['Var']
for key in STAT.keys():
    Y=STAT[key]['Var']
    print(key,sum(abs(X-Y)))

TOBS 0.0
TOBS_s20 0.0
TMAX 0.0
TMAX_s20 0.0
TMIN_s20 0.0
TMIN 0.0
SNWD_s20 0.0
SNWD 0.0
SNOW_s20 0.0
SNOW 0.0
PRCP_s20 0.0
PRCP 0.0


In [29]:
!ls -l ../../Data/Weather/STAT*

-rw-r--r--  1 yoavfreund  staff  25684524 Apr  1 14:31 ../../Data/Weather/STAT_NY.pickle


In [30]:
!gzip -f -k ../../Data/Weather/STAT*.pickle
!ls -l ../../Data/Weather/STAT*

-rw-r--r--  1 yoavfreund  staff  14948011 Apr  1 14:31 ../../Data/Weather/STAT_NY.pickle.gz


In [31]:
for state in ['NY']:
    command="aws s3  cp ../../Data/Weather/STAT_%s.pickle.gz s3://mas-dse-open/Weather/by_state/STAT_%s.pickle.gz"%(state,state)
    print(command)
    !$command

aws s3  cp ../../Data/Weather/STAT_NY.pickle.gz s3://mas-dse-open/Weather/by_state/STAT_NY.pickle.gz
upload: ../../Data/Weather/STAT_NY.pickle.gz to s3://mas-dse-open/Weather/by_state/STAT_NY.pickle.gz


In [32]:
!aws s3  ls s3://mas-dse-open/Weather/by_state/ | grep STAT

2018-04-01 14:32:56   14948011 STAT_NY.pickle.gz
2018-03-18 20:33:54   11717259 STAT_RI.pickle.gz


### Summary
* We discussed how to compute the covariance matrix and the expectation matrix when there are `nan` entries.
* The details are all in `computeStatistics`, which is defined in python files you can find in the directory `lib`